# **LAB 1 - INTRO TO GCP**



---



# **Exercise 2: Google Cloud Function with API Key**

- **Step 1.** Check to see that your API key works correctly
  - To do so: please execute the first cell of the notebook

- **Step 2.** Write and test a cloud Function in Python

- **Step 3-4.** Deploy the two functions on Google Cloud Functions

- **Step 5.** Test the deployed function from the Notebook

## **Step 1.** Check to see that your API key works correctly

- Get you API key from the [TMDB website](https://www.themoviedb.org/settings/api) and paste it inside 'YOUR_API_KEY'

- **Expected output:** Inception - this is the movie title you fetched from the movie dataset using the API

In [ ]:
import requests
import json

api_key = 'YOUR_API_KEY'
r = requests.get('https://api.themoviedb.org/3/movie/27205?api_key=' + api_key)
obj = json.loads(r.text)
movie_name = obj['title']
print(movie_name)
# print(obj['genres']) # You can also have a look at the genres associated with that movie

Inception


#### If you want to have a look at other movies, you can go on the [TMDB website](https://www.themoviedb.org/) and click on any movies for example [Dune](https://www.themoviedb.org/movie/693134-dune-part-two), the ID of that movie can be found in the URL, just copy the movie ID and replace it in the code. For example, here we would have: https://www.themoviedb.org/movie/693134-dune-part-two -> the movie ID is: 693134



## **Step 2.** Write and test a cloud Function in Python

**Goal**: The goal of these two functions are to:
- **Function 1:** Fetch a list of movies
- **Function 2:** Fetch the poster of a specific movie ID

### **Function 1:** Fetch a list of movies:

Just add your api key in "**YOUR_API_KEY**" and run the cell !


In [ ]:
#import functions_framework
import requests
from flask import jsonify

# @functions_framework.http
def get_movie_titles(): # request
    try:
        movie_database_url = 'https://api.themoviedb.org/3/discover/movie'
        api_key = 'YOUR_API_KEY'

        response = requests.get(movie_database_url, params={'api_key': api_key, 'page': 1})

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        # Check if there are results
        if 'results' in data and data['results']:
            # Filter movies that have a poster path
            movies_with_poster = [movie for movie in data['results'] if 'poster_path' in movie]

            movie_details = [{'title': movie['title'], 'id': movie['id']} for movie in movies_with_poster]

            # Return a JSON response with the list of movie details
            return {'movie_details': movie_details}
        else:
            return {'error': 'No results found.'}, 404
    except requests.RequestException as e:
        # Log the error details
        print(f"Error making API request: {e}")
        return 'Internal Server Error', 500
    except requests.exceptions.HTTPError as e:
        # Log the HTTP error details
        print(f"HTTP Error: {e}")
        return 'Internal Server Error', 500
    except json.JSONDecodeError as e:
        # Log the error details
        print(f"Error decoding JSON response: {e}")
        return 'Internal Server Error', 500


In [ ]:
# Call the function and print the result
result = get_movie_titles()
result

{'movie_details': [{'title': 'Badland Hunters', 'id': 933131},
  {'title': 'Wonka', 'id': 787699},
  {'title': 'The Marvels', 'id': 609681},
  {'title': 'Sixty Minutes', 'id': 1212073},
  {'title': 'Madame Web', 'id': 634492},
  {'title': 'Skal - Fight for Survival', 'id': 1211483},
  {'title': 'The Family Plan', 'id': 1029575},
  {'title': 'One More Shot', 'id': 1214314},
  {'title': 'Lift', 'id': 955916},
  {'title': 'Wish', 'id': 1022796},
  {'title': 'The Beekeeper', 'id': 866398},
  {'title': 'Migration', 'id': 940551},
  {'title': 'Upgraded', 'id': 1014590},
  {'title': 'Rebel Moon - Part One: A Child of Fire', 'id': 848326},
  {'title': 'The Hunger Games: The Ballad of Songbirds & Snakes',
   'id': 695721},
  {'title': 'Aquaman and the Lost Kingdom', 'id': 572802},
  {'title': 'Orion and the Dark', 'id': 1139829},
  {'title': 'Role Play', 'id': 848187},
  {'title': 'Anyone But You', 'id': 1072790},
  {'title': 'Freelance', 'id': 897087}]}

## **Step 3-4.** Deploy the two functions on Google Cloud Functions

### **Function 1:** Fetch a list of movies - copy / paste the code to Google Cloud Functions


You should also add in the **requirements.txt** the two following libraries:

functions-framework==3.*
requests==2.31.0

In [ ]:
import functions_framework
import requests
from flask import jsonify

@functions_framework.http
def get_movie_titles(request):
    try:
        movie_database_url = 'https://api.themoviedb.org/3/discover/movie'
        api_key = 'YOUR_API_KEY'

        response = requests.get(movie_database_url, params={'api_key': api_key, 'page': 1})

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        # Check if there are results
        if 'results' in data and data['results']:
            # Filter movies that have a poster path
            movies_with_poster = [movie for movie in data['results'] if 'poster_path' in movie]

            movie_details = [{'title': movie['title'], 'id': movie['id']} for movie in movies_with_poster]

            # Return a JSON response with the list of movie details
            return {'movie_details': movie_details}
        else:
            return {'error': 'No results found.'}, 404
    except requests.RequestException as e:
        # Log the error details
        print(f"Error making API request: {e}")
        return 'Internal Server Error', 500
    except requests.exceptions.HTTPError as e:
        # Log the HTTP error details
        print(f"HTTP Error: {e}")
        return 'Internal Server Error', 500
    except json.JSONDecodeError as e:
        # Log the error details
        print(f"Error decoding JSON response: {e}")
        return 'Internal Server Error', 500


### **Function 2:** Fetch the poster of a specific Movie  - copy / paste the code to Google Cloud Functions



You should also add in the **requirements.txt** the two following libraries:

- functions-framework==3.*
- requests==2.31.0

In [ ]:
import functions_framework
from flask import jsonify
import requests

api_key = 'YOUR_API_KEY'


@functions_framework.http
def get_movie_details(request):
    try:
        # Extract the movie ID from the request parameters
        movie_id = request.args.get('movie_id')

        # Make the API request using the provided movie ID
        url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Parse the JSON response
        obj = response.json()

        poster_path_prefix = 'https://image.tmdb.org/t/p/w1280/'
        poster_path = f"{poster_path_prefix}{obj.get('poster_path')}"


        # Return a JSON response with the movie details
        response_data = {'movie_name': obj["title"], 'movie_poster': poster_path}
        return jsonify(response_data)
    except requests.RequestException as e:
        # Log the error details
        print(f"Error making API request: {e}")
        return 'Internal Server Error', 500
    except requests.exceptions.HTTPError as e:
        # Log the HTTP error details
        print(f"HTTP Error: {e}")
        return 'Internal Server Error', 500
    except json.JSONDecodeError as e:
        # Log the error details
        print(f"Error decoding JSON response: {e}")
        return 'Internal Server Error', 500


## **Step 5.** Test the deployed function from the Notebook

In [ ]:
# Testing function 1:
r = requests.get('YOUR_URL')
obj = json.loads(r.text)
print(obj)

{'movie_details': [{'id': 933131, 'title': 'Badland Hunters'}, {'id': 787699, 'title': 'Wonka'}, {'id': 609681, 'title': 'The Marvels'}, {'id': 1212073, 'title': 'Sixty Minutes'}, {'id': 634492, 'title': 'Madame Web'}, {'id': 1211483, 'title': 'Skal - Fight for Survival'}, {'id': 1029575, 'title': 'The Family Plan'}, {'id': 1214314, 'title': 'One More Shot'}, {'id': 955916, 'title': 'Lift'}, {'id': 1022796, 'title': 'Wish'}, {'id': 866398, 'title': 'The Beekeeper'}, {'id': 940551, 'title': 'Migration'}, {'id': 1014590, 'title': 'Upgraded'}, {'id': 848326, 'title': 'Rebel Moon - Part One: A Child of Fire'}, {'id': 695721, 'title': 'The Hunger Games: The Ballad of Songbirds & Snakes'}, {'id': 572802, 'title': 'Aquaman and the Lost Kingdom'}, {'id': 1139829, 'title': 'Orion and the Dark'}, {'id': 848187, 'title': 'Role Play'}, {'id': 1072790, 'title': 'Anyone But You'}, {'id': 897087, 'title': 'Freelance'}]}


In [ ]:
# Testing function 2:
movie_id = '933131'
r = requests.get(f'YOUR_URL?movie_id={movie_id}')
obj = json.loads(r.text)
print(obj)

{'movie_name': 'Inception', 'movie_poster': 'https://image.tmdb.org/t/p/w1280//oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg'}
